In [1]:
!pip install git+https://github.com/firekind/athena@v0.0.1

  Cloning https://github.com/firekind/athena (to revision v0.0.1) to /tmp/pip-req-build-aph08_ay
  Running command git clone -q https://github.com/firekind/athena /tmp/pip-req-build-aph08_ay
  Running command git checkout -q 76f26a105635a8dedade28cf37be0c051bd5ddc4
  Created wheel for athena: filename=athena-0.0.1-cp36-none-any.whl size=29405 sha256=cd54c121718a54f67c961eb559d175afb5f1e8f6c853bf3f49a0bd36490d8cc8
  Stored in directory: /tmp/pip-ephem-wheel-cache-f419hh7f/wheels/c6/98/d8/85f5f37d92af24edb6c8b143a23751e6034a2c173665356fed
Successfully built athena


In [2]:
from athena import datasets, Experiment, ClassificationSolver

In [3]:
## Importing My Model
from athena.models import cifar10_v2

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
batch_size = 128 if device == 'cuda' else 64

train_loader = datasets.cifar10(download=True, batch_size=batch_size, use_default_transforms=True)
test_loader = datasets.cifar10(train=False, download=True, batch_size=batch_size, use_default_transforms=True)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
net = cifar10_v2.Cifar10V2().to(device)
summary(net, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             864
       BatchNorm2d-2           [-1, 32, 32, 32]              64
              ReLU-3           [-1, 32, 32, 32]               0
           Dropout-4           [-1, 32, 32, 32]               0
            Conv2d-5           [-1, 32, 32, 32]             288
            Conv2d-6           [-1, 32, 32, 32]           1,056
       BatchNorm2d-7           [-1, 32, 32, 32]              64
              ReLU-8           [-1, 32, 32, 32]               0
           Dropout-9           [-1, 32, 32, 32]               0
           Conv2d-10           [-1, 32, 32, 32]           9,216
      BatchNorm2d-11           [-1, 32, 32, 32]              64
             ReLU-12           [-1, 32, 32, 32]               0
          Dropout-13           [-1, 32, 32, 32]               0
           Conv2d-14           [-1, 32,

In [ ]:
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, OneCycleLR

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = StepLR(optimizer, step_size=8, gamma=0.1)

In [ ]:
def l1_loss(l1_lambda):
    def _l1_loss(y_pred, target):
        loss = F.nll_loss(y_pred, target)
        l1 = 0
        for p in model.parameters():
            l1 = l1 + p.abs().sum()
        loss = loss + l1_lambda * l1
        return loss

    return _l1_loss

In [ ]:
l1_lambda = 5e-4
exp = Experiment(
    name="New Model",
    model=model,
    solver_cls=ClassificationSolver,
    train_args=dict(
        epochs=50,
        train_loader=trainloader,
        test_loader=testloader,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        loss_fn=l1_loss(l1_lambda),
    )
)

exp.run()

=> Running experiment: New Model
Epoch: 1 / 50
1563/1563 [==============================] - 44s 28ms/step - loss: 9.2793 - accuracy: 27.8140
Test set: Average loss: 9.4941, Accuracy: 3366/10000 (33.66%)

Epoch: 2 / 50
1563/1563 [==============================] - 43s 28ms/step - loss: 6.8643 - accuracy: 44.3760
Test set: Average loss: 7.3521, Accuracy: 4935/10000 (49.35%)

Epoch: 3 / 50
1563/1563 [==============================] - 44s 28ms/step - loss: 5.8407 - accuracy: 54.0560
Test set: Average loss: 5.8899, Accuracy: 5617/10000 (56.17%)

Epoch: 4 / 50
1563/1563 [==============================] - 44s 28ms/step - loss: 5.2328 - accuracy: 59.8400
Test set: Average loss: 5.0300, Accuracy: 5901/10000 (59.01%)

Epoch: 5 / 50
1563/1563 [==============================] - 44s 28ms/step - loss: 4.2297 - accuracy: 62.8760
Test set: Average loss: 4.2838, Accuracy: 6595/10000 (65.95%)

Epoch: 6 / 50
1563/1563 [==============================] - 44s 28ms/step - loss: 4.0412 - accuracy: 65.3380
Test